In [1]:
# import all dependencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import tarfile

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# import the dataset which is compressed in the format .tar.xz
def import_dataset(path):
    dataset_path = os.path.join(path, 'personal_dataset')
    if os.path.exists(dataset_path):
        return
    os.makedirs(dataset_path, exist_ok=True)
    tgz_path = os.path.join(path, "personal_dataset.tar.xz")
    personal_dataset = tarfile.open(tgz_path)
    personal_dataset.extractall(path=path)
    personal_dataset.close()
    os.remove(tgz_path)
    

In [3]:
import_dataset('.')

In [4]:
# let's get the classes
path_of_classes = os.path.join(os.getcwd(), "personal_dataset")
classes = os.listdir(path_of_classes)
target_vec = []
for cls in classes:
    if not cls.startswith("."):
        target_vec.append(cls)
print(target_vec)

['bigyan', 'madhu', 'sonu', 'prabhat']


In [5]:
# let's get the path list for each classes
from collections import defaultdict

image_paths = defaultdict(list)
for each_class in target_vec:
    image_dir = os.path.join(path_of_classes, each_class)
    for each_image_path in os.listdir(image_dir):
        image_paths[each_class].append(os.path.join(image_dir, each_image_path))

In [6]:
# let's look at the first few examples
import matplotlib.image as mpimg

# n_examples_per_class = 2
# plt.figure(figsize=(10,5))
# for each_class in target_vec:
#     print("Class:", each_class)
#     for index, each_image in enumerate(image_paths[each_class][:n_examples_per_class]):
#         example_image = mpimg.imread(each_image)[:,:,:3]
#         plt.subplot(100 + n_examples_per_class * 10 + index + 1)
#         plt.title("{}x{}".format(example_image.shape[1], example_image.shape[0]))
#         plt.imshow(example_image)
#         plt.axis("off")
#     plt.show()

In [7]:
from scipy.misc import imresize

def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    # First, let's find the largest bounding box with the target size ratio that fits within the image
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
    # Now let's shrink this bounding box by a random factor (dividing the dimensions by a random number
    # between 1.0 and 1.0 + `max_zoom`.
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # Next, we can select a random location on the image for this bounding box.
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # Let's crop the image using the random bounding box we built.
    image = image[y0:y1, x0:x1]

    # Let's also flip the image horizontally with 50% probability:
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # Now, let's resize the image to the target dimensions.
    image = imresize(image, (target_width, target_height))
    
    # Finally, let's ensure that the colors are represented as
    # 32-bit floats ranging from 0.0 to 1.0 (for now):
    return image.astype(np.float32) / 255

In [8]:
# Define hyperparameters
width = 299
height = 299
channels = 3

In [9]:
import sys
import tarfile
from six.moves import urllib

TF_MODELS_URL = "http://download.tensorflow.org/models"
INCEPTION_V3_URL = TF_MODELS_URL + "/inception_v3_2016_08_28.tar.gz"
INCEPTION_PATH = os.path.join("datasets", "inception")
INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")

def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()

def fetch_pretrained_inception_v3(url=INCEPTION_V3_URL, path=INCEPTION_PATH):
    if os.path.exists(INCEPTION_V3_CHECKPOINT_PATH):
        return
    os.makedirs(path, exist_ok=True)
    tgz_path = os.path.join(path, "inception_v3.tgz")
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
    inception_tgz = tarfile.open(tgz_path)
    inception_tgz.extractall(path=path)
    inception_tgz.close()
    os.remove(tgz_path)

In [10]:
fetch_pretrained_inception_v3()

Downloading: 100%

In [11]:
# Now let's fetch inception V3 model
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()

In [12]:
# We just need to attach our output layer just before the ouput layer of Inception V3 model.
# inception_v3() function returns a dictionary of endpoints.
end_points

{'Conv2d_1a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_1a_3x3/Relu:0' shape=(?, 149, 149, 32) dtype=float32>,
 'Conv2d_2a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2a_3x3/Relu:0' shape=(?, 147, 147, 32) dtype=float32>,
 'Conv2d_2b_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_2b_3x3/Relu:0' shape=(?, 147, 147, 64) dtype=float32>,
 'MaxPool_3a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_3a_3x3/MaxPool:0' shape=(?, 73, 73, 64) dtype=float32>,
 'Conv2d_3b_1x1': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_3b_1x1/Relu:0' shape=(?, 73, 73, 80) dtype=float32>,
 'Conv2d_4a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/Conv2d_4a_3x3/Relu:0' shape=(?, 71, 71, 192) dtype=float32>,
 'MaxPool_5a_3x3': <tf.Tensor 'InceptionV3/InceptionV3/MaxPool_5a_3x3/MaxPool:0' shape=(?, 35, 35, 192) dtype=float32>,
 'Mixed_5b': <tf.Tensor 'InceptionV3/InceptionV3/Mixed_5b/concat:0' shape=(?, 35, 35, 256) dtype=float32>,
 'Mixed_5c': <tf.Tensor 'InceptionV3/InceptionV3/Mixed_5c/concat:0' shape=(?,

In [13]:
# we need to drop the output layer i.e logits, and attach our output layer after preLogits.
# We can drop 2 and 3rd dimension of preLogit
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

In [14]:
# Then we can add fully connected output layer to the CNN architecture.
n_outputs = len(target_vec)

with tf.name_scope("new_output_layer"):
    face_logits = tf.layers.dense(prelogits, n_outputs, name="face_logits")
    Y_proba = tf.nn.softmax(face_logits, name="Y_proba")


In [15]:
# Finally we need to add all the bits and pieces of Neural Network.

# Placeholder for y
y = tf.placeholder(tf.int32, shape=[None])

# training and loss function
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=face_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    face_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="face_logits")
    training_op = optimizer.minimize(loss, var_list=face_vars)
    
# evaluate the classifier
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(face_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# save the model.
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [16]:
[v.name for v in face_vars]

['face_logits/kernel:0', 'face_logits/bias:0']

In [17]:
# Now lets represent our classes as ints rather than strings
target_ints = {class_ids: index for index, class_ids in  enumerate(target_vec)}
print(target_ints)

{'bigyan': 0, 'madhu': 1, 'sonu': 2, 'prabhat': 3}


In [18]:
# Let's represent image and its associated class as a list

image_path_and_class = []
for image_class, image_path in image_paths.items():
    for path in image_path:
        image_path_and_class.append((path, target_ints[image_class]))

In [19]:
# let's shuffle our dataset
shuffle_ratio = 0.2
train_size = int(len(image_path_and_class) * (1 - shuffle_ratio))

np.random.shuffle(image_path_and_class)
train_set = image_path_and_class[:train_size]
test_set = image_path_and_class[train_size:]


In [20]:
print(len(train_set))
print(len(test_set))

376
94


In [21]:
# Next we we will create a function to preprocess all the images.
from random import sample

def prepare_batch(image_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(image_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    x_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return x_batch, y_batch

In [22]:
x_batch, y_batch = prepare_batch(train_set, 4)
x_batch.shape

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(4, 299, 299, 3)

In [23]:
x_test, y_test = prepare_batch(test_set, len(test_set))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [24]:
x_test.shape

(94, 299, 299, 3)

In [26]:
n_epochs = 10
batch_size = 40
n_iterations_per_epoch = len(image_path_and_class) // batch_size

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    init.run()
    inception_saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    for epoch in range(n_epochs):
        print("Epoch", epoch, end="")
        for iteration in range(n_iterations_per_epoch):
            print(".", end="")
            X_batch, y_batch = prepare_batch(image_path_and_class, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})

        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print("  Train accuracy:", acc_train)

        save_path = saver.save(sess, "./my_image_path")


INFO:tensorflow:Restoring parameters from datasets/inception/inception_v3.ckpt
Epoch 0.

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


..........  Train accuracy: 0.475
Epoch 1...........  Train accuracy: 0.525
Epoch 2...........  Train accuracy: 0.55
Epoch 3...........  Train accuracy: 0.725
Epoch 4...........  Train accuracy: 0.675
Epoch 5...........  Train accuracy: 0.575
Epoch 6...........  Train accuracy: 0.65
Epoch 7...........  Train accuracy: 0.675
Epoch 8...........  Train accuracy: 0.75
Epoch 9...........  Train accuracy: 0.8


In [27]:
n_test_batches = 10
X_test_batches = np.array_split(x_test, n_test_batches)
y_test_batches = np.array_split(y_test, n_test_batches)

with tf.Session() as sess:
    saver.restore(sess, "./my_image_path")

    print("Computing final accuracy on the test set (this will take a while)...")
    acc_test = np.mean([
        accuracy.eval(feed_dict={X: X_test_batch, y: y_test_batch})
        for X_test_batch, y_test_batch in zip(X_test_batches, y_test_batches)])
    print("Test accuracy:", acc_test)

INFO:tensorflow:Restoring parameters from ./my_image_path
Computing final accuracy on the test set (this will take a while)...
Test accuracy: 0.6711111
